A tutorial based on [An Algorithmic Introduction to Numerical Simulation of Stochastic Differential Equations][1] by Desmond J. Higham.

[1]: https://doi.org/10.1137/S0036144500378302

In [ ]:
%matplotlib notebook

import numpy as np
import numpy.random as random
random.seed(0)
import matplotlib.pyplot as plt

# Brownian Motion

Definition of Brownian motion:

1.  $W(0) = 0 \quad .$
2.  $W(t) - W(s) \sim \mathcal{N}\left(0, \sqrt{t-s}\right) \quad , \quad 0 \leq s < t \leq T \quad .$
3.  $W(t) - W(s)$ and $W(v) - W(u)$, $0 \leq s < t < u < v \leq T$, are independent.

In [ ]:
def brown(T, N, N_ens=1):
    dt = T / N
    sqrt_dt = np.sqrt(dt)

    dW = random.normal(0., sqrt_dt, (N_ens, N))
    W = np.zeros((N_ens, N + 1))
    W[:, 1:] = np.cumsum(dW, axis=1)

    return W, dW

T = 1.
N = 500
t = np.linspace(0., 1., N + 1)

N_ens = 10000
W_ens, dW_ens = brown(T, N, N_ens)

Example:

In [ ]:
fig = plt.figure(1)
ax = fig.gca()
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$W$")
for k in range(100):
    ax.plot(t, W_ens[k, :])
    
fig = plt.figure(2)
ax = fig.gca()
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"standard deviation of $W$")
std_dev = np.sqrt(np.sum(W_ens**2, axis=0) / N_ens)
ax.plot(t, np.sqrt(t), label="analytical")
ax.plot(t, std_dev, label="numerical")
ax.legend()

# Stochastic Integrals

We are going to approximate the stochastic integral $\int_0^T{h(t)\,\mathrm{d}W}$ by a Riemann sum.
In a stochastic differential equation, the term $h(t)\,\mathrm{d}W$ may represent stochastic forcing with Gaussian noise of time-dependent amplitude $h(t)$.
The approximation of stochastic integrals by Riemann sums is not unique.
Unlike for ordinary differential equations, the approximations are not equivalent as $\delta t \to 0$.

## Itô Integral

The Itô integral is defined as the left-hand sum:
\begin{equation}
\int_0^T{h(t)\,\mathrm{d}W} = \lim_{N \to \infty}{\sum_{i=0}^{N-1}{h(t_i)\left(W(t_{i+1}) - W(t_i)\right)}} \quad .
\end{equation}

For $h(t) = W(t)$, the Itô integral gives:
\begin{align}
\sum_{i=0}^{N-1}{W(t_i)\left(W(t_{i+1}) - W(t_i)\right)}
&= \frac{1}{2}\sum_{i=0}^{N-1}\left[W(t_{i+1})^2 - W(t_i)^2 - \left(W(t_{i+1}) - W(t_i)\right)^2\right] \\
&= \frac{1}{2}\sum_{i=0}^{N-1}\left[W(t_{i+1})^2 - W(t_i)^2\right] - \frac{1}{2}\sum_{i=0}^{N-1}\left(W(t_{i+1}) - W(t_i)\right)^2 \\
&= \frac{1}{2}\left[W(T)^2 - W(0)^2\right] - \frac{1}{2}\sum_{i=0}^{N-1}\left(W(t_{i+1}) - W(t_i)\right)^2 \quad , \\
\lim_{N \to \infty}{\sum_{i=0}^{N-1}{W(t_i)\left(W(t_{i+1}) - W(t_i)\right)}}
&= \frac{1}{2}\left[W(T)^2 - W(0)^2\right] - \frac{T}{2} \quad .
\end{align}

Example:

In [ ]:
ito = np.sum(W_ens[:, :-1] * dW_ens, axis=1)

fig = plt.figure(11)
ax = fig.gca()
ax.set_title("Itô integral")
ax.set_xlabel(r"$W(T)$")
ax.set_ylabel(r"$\int_0^T{W(t)\,\mathrm{d}W(t)}$")
ax.plot(np.sort(W_ens[:, -1]), 0.5 * np.sort(W_ens[:, -1])**2 - 0.5 * T, label="analytical")
ax.scatter(W_ens[:, -1], ito, label="numerical")
ax.legend()

## Stratonovich Integral

The Stratonovich integral is defined as the mid-point sum:
\begin{equation}
\int_0^T{h(t)\,\mathrm{d}W} = \lim_{N \to \infty}{\sum_{i=0}^{N-1}{h\left(\frac{t_i + t_{i+1}}{2}\right)\left(W(t_{i+1}) - W(t_i)\right)}} \quad .
\end{equation}

The conditional probability distribution of $W\left(\frac{t_i + t_{i+1}}{2}\right)$ given $W(t_i)$ and $W(t_{i+1})$ is given by
\begin{equation}
W\left(\frac{t_i + t_{i+1}}{2}\right) = \frac{W(t_i) + W(t_{i+1})}{2} + \Delta Z \quad , \quad \Delta Z \sim \mathcal{N}\left(0, \frac{\delta t}{4}\right) \quad .
\end{equation}
For $h(t) = W(t)$, the Stratonovich integral gives:
\begin{align}
\sum_{i=0}^{N-1}{W\left(\frac{t_i + t_{i+1}}{2}\right)\left(W(t_{i+1}) - W(t_i)\right)}
&= \sum_{i=0}^{N-1}{\left[\frac{W(t_i) + W(t_{i+1})}{2} + \Delta Z\right]\left(W(t_{i+1}) - W(t_i)\right)} \\
&= \frac{1}{2}\sum_{i=0}^{N-1}\left[W(t_{i+1})^2 - W(t_i)^2\right] + \Delta Z\sum_{i=0}^{N-1}\left(W(t_{i+1}) - W(t_i)\right) \\
&= \frac{1}{2}\left[W(T)^2 - W(0)^2\right] + \Delta Z\left(W(T) - W(0)\right) \quad , \\
\lim_{N \to \infty}{\sum_{i=0}^{N-1}{W\left(\frac{t_i + t_{i+1}}{2}\right)\left(W(t_{i+1}) - W(t_i)\right)}}
&= \frac{1}{2}\left[W(T)^2 - W(0)^2\right] \quad .
\end{align}

Example:

In [ ]:
strat = np.sum(0.5 * (W_ens[:, :-1] + W_ens[:, 1:]) * dW_ens, axis=1) + random.normal(0, 0.25 * T / N, N_ens)

fig = plt.figure(12)
ax = fig.gca()
ax.set_title("Stratonovich integral")
ax.set_xlabel(r"$W(T)$")
ax.set_ylabel(r"$\int_0^T{W(t)\,\mathrm{d}W(t)}$")
ax.plot(np.sort(W_ens[:, -1]), 0.5 * np.sort(W_ens[:, -1])**2, label="analytical")
ax.scatter(W_ens[:, -1], strat, label="numerical")
ax.legend()

# The Euler-Maruyama Method

A scalar, autonomous stochastic differential equation is given by
\begin{equation}
\mathrm{d}X = f(X(t))\,\mathrm{d}t + g(X(t))\,\mathrm{d}W \quad ,
\end{equation}
\begin{equation}
X(0) = X_0 \quad .
\end{equation}
If $g \equiv 0$ and $X_0$ not a random variable, the problem becomes deterministic and reduces to an ordinary differential equation.

The Euler-Maruyama method applies forward integration to the deterministic term $f(X(t))\,\mathrm{d}t$ and Itô integration to the stochastic term $g(X(t))\,\mathrm{d}W$:
\begin{equation}
X_{i+1} = X(t_i) + f(X_i)\,\mathrm{d}t + g(X_i)\left(W(t_{i+1}) - W(t_i)\right) \quad .
\end{equation}

Example:

We apply the Euler-Maruyama method to the linear stochastic differential equation
\begin{equation}
\mathrm{d}X = \lambda X(t)\,\mathrm{d}t + \mu X(t)\,\mathrm{d}W \quad ,
\end{equation}
where $\lambda$ and $\mu$ are constants.
The exact solution is given by
\begin{equation}
X(t) = X_0 \exp\left(\left[\lambda - \frac{\mu^2}{2}\right] t + \mu W(t)\right) \quad .
\end{equation}
The expected value of the solution to the stochastic differential equation is given by:
\begin{equation}
\mathrm{d}(\mathrm{E}[X]) = \lambda \mathrm{E}[X]\,\mathrm{d}t \quad ,
\end{equation}
\begin{equation}
\mathrm{E}[X] = X_0 \exp\left(\lambda t\right) \quad .
\end{equation}

In [ ]:
def lsde(T, N, N_ens, X_0, W, lamb, mu):
    return X_0 * np.exp((lamb - 0.5 * mu**2) * np.linspace(0., T, N + 1) + mu * W)

def lsde_em(T, N, N_ens, X_0, W, lamb, mu, stride=1):
    N_e = N // stride
    dt_e = T / N_e
    
    res = np.zeros((N_ens, N_e  + 1))
    res[:, 0] = X_0
    for i in range(N_e):
        res[:, i + 1] = res[:, i] + lamb * res[:, i] * dt_e + mu * res[:, i] * (W[:, (i + 1) * stride] - W[:, i * stride])
    
    return res

lamb = 2.
mu = 1.
em = lsde(T, N, N_ens, 1., W_ens, lamb, mu)

em_1 = lsde_em(T, N, N_ens, 1., W_ens, lamb, mu)
em_2 = lsde_em(T, N, N_ens, 1., W_ens, lamb, mu, 2)
em_5 = lsde_em(T, N, N_ens, 1., W_ens, lamb, mu, 5)

for i in range(3):
    fig = plt.figure(20 + i)
    ax = fig.gca()
    ax.set_title("Euler-Maruyama method (sample {})".format(i))
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$X$")
    ax.plot(np.linspace(0., T, N + 1), em[i, :], label="analytical")
    ax.plot(np.linspace(0., T, N + 1), em_1[i, :], label="numerical (dt={})".format(T / N))
    ax.plot(np.linspace(0., T, N // 2 + 1), em_2[i, :], label="numerical (dt={})".format(2 * T / N))
    ax.plot(np.linspace(0., T, N // 5 + 1), em_5[i, :], label="numerical (dt={})".format(5 * T / N))
    ax.legend()

In [ ]:
fig = plt.figure(30)
ax = fig.gca()
ax.set_title("Euler-Maruyama method (expected value)".format(i))
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$X$")
ax.plot(np.linspace(0., T, N + 1), np.exp(lamb * np.linspace(0., T, N + 1)), label="analytical")
ax.plot(np.linspace(0., T, N + 1), np.mean(em, axis=0), label="analytical (average)")
ax.plot(np.linspace(0., T, N + 1), np.mean(em_1, axis=0), label="numerical (average, dt={})".format(T / N))
ax.plot(np.linspace(0., T, N // 2 + 1), np.mean(em_2, axis=0), label="numerical (average, dt={})".format(2 * T / N))
ax.plot(np.linspace(0., T, N // 5 + 1), np.mean(em_5, axis=0), label="numerical (average, dt={})".format(5 * T / N))
ax.legend()

## Strong and Weak Convergence

A stochastic method has strong order of convergence $\gamma$ if
\begin{equation}
\mathrm{E}[||X_n - X(t_n)||] \sim \mathcal{O}((\Delta t)^\gamma) \quad .
\end{equation}
A stochastic method has weak order of convergence $\alpha$ if
\begin{equation}
||\mathrm{E}[X_n] - \mathrm{E}[X(t_n)]|| \sim \mathcal{O}((\Delta t)^\alpha) \quad .
\end{equation}

In [ ]:
delta_t = [1 * T / N, 2 * T / N, 5 * T / N]

error_strong = []
error_strong.append(np.mean(np.abs(em_1 - em), axis=0)[-1])
error_strong.append(np.mean(np.abs(em_2 - em[:, ::2]), axis=0)[-1])
error_strong.append(np.mean(np.abs(em_5 - em[:, ::5]), axis=0)[-1])

em_mean = np.mean(em, axis=0)
error_weak = []
error_weak.append(np.abs(np.mean(em_1, axis=0) - em_mean)[-1])
error_weak.append(np.abs(np.mean(em_2, axis=0) - em_mean[::2])[-1])
error_weak.append(np.abs(np.mean(em_5, axis=0) - em_mean[::5])[-1])

fig = plt.figure(40)
ax = fig.gca()
ax.set_title("Stochastic convergence")
ax.set_xlabel(r"$\Delta t$")
ax.set_ylabel(r"error")
ax.plot(delta_t, error_strong, label="strong")
ax.plot(delta_t, error_weak, label="weak")
ax.plot(delta_t, np.array(delta_t)**0.5 * error_strong[1] / delta_t[1]**0.5, label=r"strong ($C (\Delta t)^{0.5}$)")
ax.plot(delta_t, np.array(delta_t) * error_weak[1] / delta_t[1], label=r"weak ($C \Delta t$)")
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')